In [111]:
import pandas as pd
import os

# load the file path to your raw data
Environmental_data = os.path.join("C:/Users/emman/Downloads/PM2.5_Pollution_Data-Public/Data/Cleaned_Air_Quality_Data_AT(in).csv")

# 📥 Load the data into a pandas DataFrame
df = pd.read_csv(Environmental_data)

#Preview the shape and head to confirm it loaded correctly
print("Dataset successfully loaded.")
print(f"Shape of dataset: {df.shape}")  # (rows, columns)
print(df.head())  # Display first 5 rows


Dataset successfully loaded.
Shape of dataset: (443, 39)
   numeric_area_code composite_area_code Local Authority  PM2.5 2012 (total)  \
0                1.0           E07000223            Adur             10.6389   
1                2.0           E07000026       Allerdale              6.5756   
2                4.0           E07000032    Amber Valley             10.4766   
3                5.0           E07000224            Arun             10.0845   
4                6.0           E07000170        Ashfield             11.1841   

   PM2.5 2012 (non-anthropogenic)  PM2.5 2012 (anthropogenic)  \
0                          2.2917                      8.3472   
1                          1.9619                      4.6137   
2                          2.0689                      8.4077   
3                          2.2713                      7.8132   
4                          2.0643                      9.1198   

   PM2.5 2013 (total)  PM2.5 2013 (non-anthropogenic)  \
0             

In [112]:
# ---------------------------------------------
# MISSING VALUE SUMMARY
# ---------------------------------------------

#Count total missing values per column
missing_counts = df.isnull().sum()

#Calculate % missing for each column
missing_percentage = (missing_counts / len(df)) * 100

#Combine into a summary DataFrame
missing_summary = pd.DataFrame({
    'Missing_Count': missing_counts,
    'Missing_Percentage': missing_percentage.round(2)
}).sort_values(by='Missing_Percentage', ascending=False)

# Output top missing columns
print("\nMissing Value Summary:")
print(missing_summary.head(39))  # View top 39 columns with missing values

#Save summary to outputs
#missing_summary.to_csv("../outputs/missing_summary.csv")



Missing Value Summary:
                                Missing_Count  Missing_Percentage
composite_area_code                       129               29.12
PM2.5 2023 (non-anthropogenic)             82               18.51
PM2.5 2023 (total)                         82               18.51
PM2.5 2023 (anthropogenic)                 82               18.51
PM2.5 2022 (non-anthropogenic)             69               15.58
PM2.5 2022 (total)                         69               15.58
PM2.5 2021 (total)                         69               15.58
PM2.5 2022 (anthropogenic)                 69               15.58
PM2.5 2021 (non-anthropogenic)             69               15.58
PM2.5 2021 (anthropogenic)                 69               15.58
PM2.5 2020 (anthropogenic)                 60               13.54
PM2.5 2019 (non-anthropogenic)             60               13.54
PM2.5 2020 (total)                         60               13.54
PM2.5 2019 (anthropogenic)                 60       

In [113]:
# RULE 1: Rename columns for consistency
# Rename key columns for ease of use
df.rename(columns={
    'numeric_area_code': 'Numeric Area Code',
    'composite_area_code': 'Area Code',
    'Local Authority': 'Local Authority Name'
}, inplace=True)
print(df.head())  # Display first 5 rows


   Numeric Area Code  Area Code Local Authority Name  PM2.5 2012 (total)  \
0                1.0  E07000223                 Adur             10.6389   
1                2.0  E07000026            Allerdale              6.5756   
2                4.0  E07000032         Amber Valley             10.4766   
3                5.0  E07000224                 Arun             10.0845   
4                6.0  E07000170             Ashfield             11.1841   

   PM2.5 2012 (non-anthropogenic)  PM2.5 2012 (anthropogenic)  \
0                          2.2917                      8.3472   
1                          1.9619                      4.6137   
2                          2.0689                      8.4077   
3                          2.2713                      7.8132   
4                          2.0643                      9.1198   

   PM2.5 2013 (total)  PM2.5 2013 (non-anthropogenic)  \
0             10.6921                          2.2585   
1              7.7929                 

In [114]:
# RULE 2: Drop rows with >80% missing PM2.5 values

pm25_columns = [col for col in df.columns if 'PM2.5' in col and 'Missing_Count' not in col]#Identify all PM2.5-related columns

# Count how many PM2.5 values are missing per row
df['PM2.5_Missing_Count'] = df[pm25_columns].isnull().sum(axis=1)
print(df['PM2.5_Missing_Count'])

#print(df[['PM2.5_Missing_Count']].sort_values(by='PM2.5_Missing_Count', ascending=False))#to view the rows with the most missing PM2.5 values

#print(df['PM2.5_Missing_Count'].value_counts())#to see how many rows have how many missing values

# Drop rows where more than 80% of PM2.5 columns are missing
pm25_threshold = int(0.8 * len(pm25_columns))#create a threshold saying if 80% of length of row for PM2.5 columns e.g. 0.8 x 39 = threshold = 31.2
df = df[df['PM2.5_Missing_Count'] <= pm25_threshold]#keep only those rows where the number if missing PM2.5 columns is less than or equal to 31.2

# Remove temporary helper column
df.drop(columns='PM2.5_Missing_Count', inplace=True)

print(df.shape)         # Should show fewer rows if any were dropped
print(df.columns)       # Should list all columns, without helper columns
print(pm25_columns)

#We used the pm25_threshold value to remove 32 rows of PM2.5 data with >80% missing PM values

0       0
1       3
2       0
3       0
4       0
       ..
438    27
439    33
440    33
441    33
442    33
Name: PM2.5_Missing_Count, Length: 443, dtype: int64
(411, 39)
Index(['Numeric Area Code', 'Area Code', 'Local Authority Name',
       'PM2.5 2012 (total)', 'PM2.5 2012 (non-anthropogenic)',
       'PM2.5 2012 (anthropogenic)', 'PM2.5 2013 (total)',
       'PM2.5 2013 (non-anthropogenic)', 'PM2.5 2013 (anthropogenic)',
       'PM2.5 2014 (total)', 'PM2.5 2014 (non-anthropogenic)',
       'PM2.5 2014 (anthropogenic)', 'PM2.5 2015 (total)',
       'PM2.5 2015 (non-anthropogenic)', 'PM2.5 2015 (anthropogenic)',
       'PM2.5 2016 (total)', 'PM2.5 2016 (non-anthropogenic)',
       'PM2.5 2016 (anthropogenic)', 'PM2.5 2017 (total)',
       'PM2.5 2017 (non-anthropogenic)', 'PM2.5 2017 (anthropogenic)',
       'PM2.5 2018 (total)', 'PM2.5 2018 (non-anthropogenic)',
       'PM2.5 2018 (anthropogenic)', 'PM2.5 2019 (total)',
       'PM2.5 2019 (non-anthropogenic)', 'PM2.5 2019 (anthrop

In [124]:
# RULE 3: Impute missing Area Codes using Majority Mapping

# Count and display missing Area Code values
missing_before = df['Area Code'].isnull().sum()
print(f"Missing Area Codes before imputation: {missing_before}")

# Backup the original Area Code column
df['Original Area Code'] = df['Area Code']

# Compute most frequent Area Code (mode) for each Local Authority
area_code_map = df.groupby('Local Authority Name')['Area Code'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)# This creates a series where each Local Authority Name is mapped to the most frequent (non-null) Area Code.

# Fill missing Area Codes using the majority mapping
df['Area Code'] = df.apply(
    lambda row: area_code_map[row['Local Authority Name']] if pd.isnull(row['Area Code']) else row['Area Code'],
    axis=1
)

# Compare original vs. updated to see which rows got filled by imputation
filled_rows = df[df['Original Area Code'].isnull() & df['Area Code'].notnull()]
print("Rows with Area Code filled by imputation:")
print(filled_rows[['Local Authority Name', 'Original Area Code', 'Area Code']])
print(f"Filled Area Codes count: {len(filled_rows)}")

#Check missing Area Codes after imputation
missing_after = df['Area Code'].isnull().sum()
print(f"Missing Area Codes after imputation: {missing_after}")

# Look at Local Authorities that still have missing Area Codes
remaining_missing = df[df['Area Code'].isnull()]
print("Local Authorities still missing Area Codes:")
print(remaining_missing['Local Authority Name'].unique())

#Save the rows with missing Area Code into a separate DataFrame before dropping them
dropped_rows = remaining_missing.copy()

# Drop rows with missing Area Codes
df = df[~df['Area Code'].isnull()]

#Output info about dropped rows
print("Dropped rows with missing Area Codes:")
print(dropped_rows[['Local Authority Name', 'Original Area Code']])
print(f"Total dropped rows: {len(dropped_rows)}")


Missing Area Codes before imputation: 0
Rows with Area Code filled by imputation:
Empty DataFrame
Columns: [Local Authority Name, Original Area Code, Area Code]
Index: []
Filled Area Codes count: 0
Missing Area Codes after imputation: 0
Local Authorities still missing Area Codes:
[]
Dropped rows with missing Area Codes:
Empty DataFrame
Columns: [Local Authority Name, Original Area Code]
Index: []
Total dropped rows: 0


In [118]:
#RULE 4: Remove duplicates

# Save the original dataframe before dropping duplicates
before_dropping_duplicates = df.copy()

# Drop duplicates based on 'Local Authority Name', keep the first occurrence
df = df.drop_duplicates(subset=['Local Authority Name'])#Remove duplicates based on 'Local Authority Name'

# Find duplicates that were dropped:
# These are rows where 'Local Authority Name' appeared more than once in the original df,
# excluding the first occurrence.

# Get all duplicated names except the first occurrence
duplicates_dropped = before_dropping_duplicates[before_dropping_duplicates.duplicated(subset=['Local Authority Name'], keep='first')]

print("Duplicates dropped based on 'Local Authority Name':")
print(duplicates_dropped[['Local Authority Name']])

print(f"Total duplicates dropped: {len(duplicates_dropped)}")


Duplicates dropped based on 'Local Authority Name':
Empty DataFrame
Columns: [Local Authority Name]
Index: []
Total duplicates dropped: 0


In [119]:
# RULE 11: Standardise string fields

# Convert Area Code to uppercase and remove extra spaces
df['Area Code'] = df['Area Code'].astype(str).str.upper().str.strip()

# Format text fields consistently
df['Local Authority Name'] = df['Local Authority Name'].astype(str).str.title().str.strip()



In [125]:
# FINAL SUMMARY
print("\nCleaning complete.")
print("Cleaned dataset shape:", df.shape)
print("Remaining nulls per column:\n", df.isnull().sum())
df = df.drop(columns=['Original Area Code'])#drop the Original Area Code column 
print("Shape after dropping 'Original Area Code':", df.shape)



Cleaning complete.
Cleaned dataset shape: (309, 40)
Remaining nulls per column:
 Numeric Area Code                  3
Area Code                          0
Local Authority Name               0
PM2.5 2012 (total)                13
PM2.5 2012 (non-anthropogenic)    13
PM2.5 2012 (anthropogenic)        13
PM2.5 2013 (total)                13
PM2.5 2013 (non-anthropogenic)    13
PM2.5 2013 (anthropogenic)        13
PM2.5 2014 (total)                12
PM2.5 2014 (non-anthropogenic)    12
PM2.5 2014 (anthropogenic)        12
PM2.5 2015 (total)                12
PM2.5 2015 (non-anthropogenic)    12
PM2.5 2015 (anthropogenic)        12
PM2.5 2016 (total)                 9
PM2.5 2016 (non-anthropogenic)     9
PM2.5 2016 (anthropogenic)         9
PM2.5 2017 (total)                 9
PM2.5 2017 (non-anthropogenic)     9
PM2.5 2017 (anthropogenic)         9
PM2.5 2018 (total)                 9
PM2.5 2018 (non-anthropogenic)     9
PM2.5 2018 (anthropogenic)         9
PM2.5 2019 (total)            

In [128]:
print(df.head(450))  # Display first 5 rows


     Numeric Area Code  Area Code       Local Authority Name  \
0                  1.0  E07000223                       Adur   
1                  2.0  E07000026                  Allerdale   
2                  4.0  E07000032               Amber Valley   
3                  5.0  E07000224                       Arun   
4                  6.0  E07000170                   Ashfield   
..                 ...        ...                        ...   
434              462.0  E07000245               West Suffolk   
435              463.0  E07000246  Somerset West And Taunton   
436                NaN  E06000060            Buckinghamshire   
437                NaN  E06000061     North Northamptonshire   
438                NaN  E06000062      West Northamptonshire   

     PM2.5 2012 (total)  PM2.5 2012 (non-anthropogenic)  \
0               10.6389                          2.2917   
1                6.5756                          1.9619   
2               10.4766                          2.068

In [129]:
df.to_csv("Cleaned_Environmental_Data.csv", index=False)
